In [8]:
from unstructured import partition
from unstructured.partition.html import partition_html
from unstructured.chunking.title import chunk_by_title
import os
from tqdm import tqdm
import json

In [ ]:
def partition_saved_html_with_strategy(html_file_path):

    try:
        # Check if file exists
        if not os.path.exists(html_file_path):
            print(f"Error: File {html_file_path} does not exist")
            return None
        
        # Partition the HTML file with section-based chunking
        elements = partition_html(
            filename=html_file_path,
            infer_table_structure=True,
            strategy="hi_res",
            chunking_strategy="by_title",
            include_page_breaks=True,
            # Section-based chunking parameters
            max_characters=10000,  # Adjust based on your needs
            combine_text_under_n_chars= 100,  # Combine small text fragments
            )
        
        # print(f"Successfully partitioned {html_file_path}")
        # print(f"Found {len(elements)} elements")
        
        return elements
        
    except Exception as e:
        print(f"Error partitioning HTML file: {e}")
        return None

In [43]:
EXCLUDED_SECTIONS = [
    "references", "reference", "navigation", "navigation menu", "history",
    "see also", "notes", "external links", "trivia", "gallery",
    "quotes", "bugs", "changelog", "patch history", "credits",
    "footnotes", "footer", "acknowledgements", "disclaimer"
]

def should_exclude_section(section_title: str) -> bool:
    if not section_title:
        return False
    normalized = section_title.strip().lower()
    return any(excluded in normalized for excluded in EXCLUDED_SECTIONS)

In [44]:
def partition_files(files):
    texts = []
    tables = []

    for file in tqdm(files):
        # print (file)
        elements = partition_saved_html_with_strategy("raw_html/" + file)

        if not elements:
            continue

        for i in elements:
            page_title = file.split(".")[0]
            section_title = page_title

            if "CompositeElement" in str(type(i)):
                for j in i.metadata.orig_elements:
                    if "Title" in str(type(j)):
                        section_title = j.text.strip()
                    elif "Table" in str(type(j)):
                        if not should_exclude_section(section_title):
                            tables.append({
                                "page_title": page_title,
                                "section_title": section_title,
                                "table": j,
                            })
                    else:
                        if not should_exclude_section(section_title):
                            text_content = j.text.strip() if hasattr(j, "text") else str(j).strip()
                            if len(text_content) >= 100:
                                texts.append({
                                    "page_title": page_title,
                                    "section_title": section_title,
                                    "text": text_content,
                                })

            elif "Table" in str(type(i)):
                if not should_exclude_section(section_title):
                    tables.append({
                        "page_title": page_title,
                        "section_title": section_title,
                        "table": i,
                    })
            else:
                if not should_exclude_section(section_title):
                    text_content = i.text.strip() if hasattr(i, "text") else str(i).strip()
                    if len(text_content) >= 100:
                        texts.append({
                            "page_title": page_title,
                            "section_title": section_title,
                            "text": text_content,
                        })

    return texts, tables



In [45]:
from pathlib import Path

raw_html_path = Path('raw_html')
file_names = [f.name for f in raw_html_path.iterdir() if f.is_file()]
# file_names = file_names[100:110]

In [46]:
texts, tables = partition_files(file_names)

100%|██████████| 2683/2683 [05:00<00:00,  8.93it/s]


In [58]:
len(texts), len(tables)

(19039, 12010)

In [50]:
import tiktoken
encoding = tiktoken.encoding_for_model("text-embedding-3-small")
txt= tables[0]["table"].metadata.text_as_html
tokens = encoding.encode(txt)
len(tokens)

96

In [51]:
texts

[{'page_title': '%271000_Years_From_Now%27',
  'section_title': "'1000 Years From Now'",
  'text': '\'1000 Years From Now\' is a piece of furniture that hangs on a wall. It rotates into Famous Painter Lupini\'s stock on Winter 16 during the Night Market starting in year 2, and reappears on Winter 16 every 3 years. It can be purchased for data-sort-value="1200">'},
 {'page_title': '%271000_Years_From_Now%27',
  'section_title': '%271000_Years_From_Now%27',
  'text': 'Retrieved from "https://stardewvalleywiki.com/mediawiki/index.php?title=%271000_Years_From_Now%27&oldid=188796"'},
 {'page_title': '%27Abstract%27',
  'section_title': '%27Abstract%27',
  'text': 'Retrieved from "https://stardewvalleywiki.com/mediawiki/index.php?title=%27Abstract%27&oldid=188069"'},
 {'page_title': '%27A_Night_On_Eco-Hill%27',
  'section_title': "'A Night On Eco-Hill'",
  'text': "'A Night On Eco-Hill' is a piece of furniture that hangs on a wall. It can be obtained only by donating 20 items to the Museum."

In [52]:
tables

[{'page_title': '%271000_Years_From_Now%27',
  'section_title': "'1000 Years From Now'",
  'table': <unstructured.documents.elements.Table at 0x14f620a0130>},
 {'page_title': '%271000_Years_From_Now%27',
  'section_title': '%271000_Years_From_Now%27',
  'table': <unstructured.documents.elements.TableChunk at 0x14f620a1860>},
 {'page_title': '%271000_Years_From_Now%27',
  'section_title': '%271000_Years_From_Now%27',
  'table': <unstructured.documents.elements.TableChunk at 0x14f620a18d0>},
 {'page_title': '%27Abstract%27',
  'section_title': "'Abstract'",
  'table': <unstructured.documents.elements.Table at 0x14f620a0280>},
 {'page_title': '%27Abstract%27',
  'section_title': '%27Abstract%27',
  'table': <unstructured.documents.elements.TableChunk at 0x14f620a24a0>},
 {'page_title': '%27Abstract%27',
  'section_title': '%27Abstract%27',
  'table': <unstructured.documents.elements.TableChunk at 0x14f620a2510>},
 {'page_title': '%27A_Night_On_Eco-Hill%27',
  'section_title': "'A Night On

In [54]:
texts[0]

{'page_title': '%271000_Years_From_Now%27',
 'section_title': "'1000 Years From Now'",
 'text': '\'1000 Years From Now\' is a piece of furniture that hangs on a wall. It rotates into Famous Painter Lupini\'s stock on Winter 16 during the Night Market starting in year 2, and reappears on Winter 16 every 3 years. It can be purchased for data-sort-value="1200">'}

In [57]:
tables[0]

{'page_title': '%271000_Years_From_Now%27',
 'section_title': "'1000 Years From Now'",
 'table': <unstructured.documents.elements.Table at 0x14f620a0130>}

In [94]:
summarize_prompt = """
You are an assistant tasked with summarizing the tables. 
Give a concise summary of the table.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table chunk: {element}
"""

In [59]:
test_element = partition_saved_html_with_strategy("raw_html/Weapons.html")

In [61]:
for e in test_element:
    print(e.metadata.orig_elements)

[<unstructured.documents.elements.Title object at 0x0000014F3B963D20>, <unstructured.documents.elements.Text object at 0x0000014F3F1446E0>, <unstructured.documents.elements.NarrativeText object at 0x0000014F3F1451D0>, <unstructured.documents.elements.NarrativeText object at 0x0000014F3F144E50>, <unstructured.documents.elements.NarrativeText object at 0x0000014F3F144280>, <unstructured.documents.elements.NarrativeText object at 0x0000014F3F145A20>]
[<unstructured.documents.elements.Title object at 0x0000014F3F145D30>, <unstructured.documents.elements.ListItem object at 0x0000014F3F1458D0>, <unstructured.documents.elements.ListItem object at 0x0000014F3F145EF0>, <unstructured.documents.elements.ListItem object at 0x0000014F3F147C40>, <unstructured.documents.elements.ListItem object at 0x0000014F3F145FD0>, <unstructured.documents.elements.ListItem object at 0x0000014F3F146270>, <unstructured.documents.elements.ListItem object at 0x0000014F3F147CB0>, <unstructured.documents.elements.ListIt

In [76]:
elements[3].metadata.orig_elements[2].metadata.text_as_html

'<table><tr><td>Image</td><td>Name</td><td>Level</td><td>Description</td><td>Damage</td><td>Critical Strike Chance</td><td>Stats</td><td>Location</td><td>Purchase Price</td><td>Sell Price</td></tr><tr><td/><td>Rusty Sword</td><td>1</td><td>A rusty, dull old sword.</td><td>2-5</td><td>.02</td><td>N/A</td><td>The Mines entrance</td><td>data-sort-value=&quot;250&quot;&gt; 250g</td><td>data-sort-value=&quot;50&quot;&gt; 50g</td></tr><tr><td/><td>Steel Smallsword</td><td>1</td><td>A standard metal blade.</td><td>4-8</td><td>.02</td><td>Speed (+2)</td><td>The Mines floor 20 chest reward</td><td>data-sort-value=&quot;750&quot;&gt; 750g</td><td>data-sort-value=&quot;50&quot;&gt; 50g</td></tr><tr><td/><td>Wooden Blade</td><td>1</td><td>Not bad for a piece of carved wood.</td><td>3-7</td><td>.02</td><td>N/A</td><td>The Mines floor 10 chest reward (remixed mine rewards only) Adventurer&#x27;s Guild</td><td>data-sort-value=&quot;250&quot;&gt; 250g</td><td>data-sort-value=&quot;50&quot;&gt; 50g</td

In [89]:
swords_table ="'<table><tr><td>Image</td><td>Name</td><td>Level</td><td>Description</td><td>Damage</td><td>Critical Strike Chance</td><td>Stats</td><td>Location</td><td>Purchase Price</td><td>Sell Price</td></tr><tr><td/><td>Rusty Sword</td><td>1</td><td>A rusty, dull old sword.</td><td>2-5</td><td>.02</td><td>N/A</td><td>The Mines entrance</td><td>data-sort-value=&quot;250&quot;&gt; 250g</td><td>data-sort-value=&quot;50&quot;&gt; 50g</td></tr><tr><td/><td>Steel Smallsword</td><td>1</td><td>A standard metal blade.</td><td>4-8</td><td>.02</td><td>Speed (+2)</td><td>The Mines floor 20 chest reward</td><td>data-sort-value=&quot;750&quot;&gt; 750g</td><td>data-sort-value=&quot;50&quot;&gt; 50g</td></tr><tr><td/><td>Wooden Blade</td><td>1</td><td>Not bad for a piece of carved wood.</td><td>3-7</td><td>.02</td><td>N/A</td><td>The Mines floor 10 chest reward (remixed mine rewards only) Adventurer&#x27;s Guild</td><td>data-sort-value=&quot;250&quot;&gt; 250g</td><td>data-sort-value=&quot;50&quot;&gt; 50g</td></tr><tr><td/><td>Pirate&#x27;s Sword</td><td>2</td><td>It looks like a pirate owned this once.</td><td>8-14</td><td>.02</td><td>Speed (+2)</td><td>The Mines floor 50 chest reward (remixed mine rewards only) Adventurer&#x27;s Guild (after unlocking the Mines floor 30)</td><td>data-sort-value=&quot;850&quot;&gt; 850g</td><td>data-sort-value=&quot;100&quot;&gt; 100g</td></tr><tr><td/><td>Silver Saber</td><td>2</td><td>Plated with silver to deter rust.</td><td>8-15</td><td>.02</td><td>Defense (+1)</td><td>The Mines floor 50 chest reward (remixed mine rewards only) Adventurer&#x27;s Guild (after unlocking the Mines floor 20)</td><td>data-sort-value=&quot;750&quot;&gt; 750g</td><td>data-sort-value=&quot;100&quot;&gt; 100g</td></tr><tr><td/><td>Cutlass</td><td>3</td><td>A finely crafted blade.</td><td>9-17</td><td>.02</td><td>Speed (+2)</td><td>The Mines floor 60 chest reward (remixed mine rewards only) Adventurer&#x27;s Guild (after unlocking the Mines floor 25)</td><td>data-sort-value=&quot;1500&quot;&gt; 1,500g</td><td>data-sort-value=&quot;150&quot;&gt; 150g</td></tr><tr><td/><td>Forest Sword</td><td>3</td><td>Made powerful by forest magic.</td><td>8-18</td><td>.02</td><td>Speed (+2) Defense (+1)</td><td>The Mines (Floors 20-60)</td><td>N/A</td><td>data-sort-value=&quot;150&quot;&gt; 150g</td></tr><tr><td/><td>Iron Edge</td><td>3</td><td>A heavy broadsword.</td><td>12-25</td><td>.02</td><td>Speed (−2) Defense (+1) Weight (+3)</td><td>The Mines floor 60 chest reward (remixed mine rewards only) The Mines (Floors 41-59)</td><td>N/A</td><td>data-sort-value=&quot;150&quot;&gt; 150g</td></tr><tr><td/><td>Insect Head</td><td>6</td><td>Not very pleasant to wield.</td><td>20-30</td><td>.04</td><td>Speed (+2) Crit. Chance (+2)</td><td>Adventurer&#x27;s Guild reward for killing 80 Bugs</td><td>data-sort-value=&quot;10000&quot;&gt; 10,000g</td><td>data-sort-value=&quot;200&quot;&gt; 200g</td></tr><tr><td/><td>Bone Sword</td><td>5</td><td>A very light piece of sharpened bone.</td><td>20-30</td><td>.02</td><td>Speed (+4) Weight (+2)</td><td>The Mines floor 90 chest reward (remixed mine rewards only) Adventurer&#x27;s Guild (after unlocking the Mines floor 75) Skeleton drops</td><td>data-sort-value=&quot;6000&quot;&gt; 6,000g</td><td>data-sort-value=&quot;250&quot;&gt; 250g</td></tr><tr><td/><td>Claymore</td><td>5</td><td>It&#x27;s really heavy.</td><td>20-32</td><td>.02</td><td>Speed (−4) Defense (+2) Weight (+3)</td><td>The Mines floor 80 chest reward (remixed mine rewards only) Adventurer&#x27;s Guild (after unlocking the Mines floor 45)</td><td>data-sort-value=&quot;2000&quot;&gt; 2,000g</td><td>data-sort-value=&quot;250&quot;&gt; 250g</td></tr><tr><td/><td>Neptune&#x27;s Glaive</td><td>5</td><td>An heirloom from beyond the Gem Sea.</td><td>18-35</td><td>.02</td><td>Speed (−1) Defense (+2) Weight (+4)</td><td>Fishing Treasure Chests (Fishing Level 2+)</td><td>N/A</td><td>data-sort-value=&quot;250&quot;&gt; 250g</td></tr><tr><td/><td>Templar&#x27;s Blade</td><td>5</td><td>It once belonged to an honorable knight.</td><td>22-29</td><td>0</td><td>Defense (+1)</td><td>The Mines floor 80 chest reward (remixed mine rewards only) Adventurer&#x27;s Guild (after unlocking the Mines floor 55)</td><td>data-sort-value=&quot;4000&quot;&gt; 4,000g</td><td>data-sort-value=&quot;250&quot;&gt; 250g</td></tr><tr><td/><td>Obsidian Edge</td><td>6</td><td>It&#x27;s incredibly sharp.</td><td>30-45</td><td>.02</td><td>Speed (−1) Crit. Power (+10)</td><td>The Mines floor 90 chest reward</td><td>N/A</td><td>data-sort-value=&quot;300&quot;&gt; 300g</td></tr><tr><td/><td>Ossified Blade</td><td>6</td><td>A large, sharp blade formed from bone.</td><td>26-42</td><td>.02</td><td>Speed (−2) Defense (+1) Weight (+2)</td><td>The Mines floor 90 chest reward (remixed mine rewards only) Mystery Box</td><td>N/A</td><td>data-sort-value=&quot;300&quot;&gt; 300g</td></tr><tr><td/><td>Holy Blade</td><td>7</td><td>It feels hopeful to wield.</td><td>20-27</td><td>.02</td><td>Speed (+4) Defense (+2) Crusader</td><td>The Mines after level 80</td><td>N/A</td><td>data-sort-value=&quot;350&quot;&gt; 350g</td></tr><tr><td/><td>Tempered Broadsword</td><td>7</td><td>It looks like it could withstand anything.</td><td>29-44</td><td>.02</td><td>Speed (−3) Defense (+3) Weight (+3)</td><td>The Mines floor 90 chest reward (remixed mine rewards only) Skull Cavern</td><td>N/A</td><td>data-sort-value=&quot;350&quot;&gt; 350g</td></tr><tr><td/><td>Yeti Tooth</td><td>7</td><td>It&#x27;s icy cold to the touch.</td><td>26-42</td><td>.02</td><td>Defense (+4) Crit. Power (+10)</td><td>The Mines (Floors 81-99)</td><td>N/A</td><td>data-sort-value=&quot;350&quot;&gt; 350g</td></tr><tr><td/><td>Steel Falchion</td><td>8</td><td>Light and powerful.</td><td>28-46</td><td>.02</td><td>Speed (+4) Crit. Power (+20)</td><td>The Mines floor 110 chest reward (remixed mine rewards only) The Mines (Floors 101+) Skull Cavern Adventurer&#x27;s Guild (after unlocking the Mines floor 90)</td><td>data-sort-value=&quot;9000&quot;&gt; 9,000g</td><td>data-sort-value=&quot;400&quot;&gt; 400g</td></tr><tr><td/><td>Dark Sword</td><td>9</td><td>It&#x27;s glowing with vampire energy.</td><td>30-45</td><td>.04</td><td>Speed (−5) Crit. Chance (+2) Weight (+5) Vampiric (9%)</td><td>Haunted Skull drop</td><td>N/A</td><td>data-sort-value=&quot;450&quot;&gt; 450g</td></tr><tr><td/><td>Lava Katana</td><td>10</td><td>A powerful blade forged in a pool of churning lava.</td><td>55-64</td><td>.015</td><td>Defense (+3) Crit. Power (+25) Weight (+3)</td><td>Adventurer&#x27;s Guild (after unlocking the Mines floor 120)</td><td>data-sort-value=&quot;25000&quot;&gt; 25,000g</td><td>data-sort-value=&quot;500&quot;&gt; 500g</td></tr><tr><td/><td>Dragontooth Cutlass</td><td>13</td><td>The blade was forged from a magical tooth.</td><td>75-90</td><td>.02</td><td>Crit. Power (+50)</td><td>Volcano Dungeon Chest</td><td>N/A</td><td>data-sort-value=&quot;650&quot;&gt; 650g</td></tr><tr><td/><td>Dwarf Sword</td><td>13</td><td>It&#x27;s ancient, but the blade never dulls.</td><td>65-75</td><td>.02</td><td>Speed (+2) Defense (+4)</td><td>Volcano Dungeon Chest</td><td>N/A</td><td>data-sort-value=&quot;650&quot;&gt; 650g</td></tr><tr><td/><td>Galaxy Sword</td><td>13</td><td>It&#x27;s unlike anything you&#x27;ve ever seen.</td><td>60-80</td><td>.02</td><td>Speed (+4)</td><td>The Desert (bring a Prismatic Shard to the Three Pillars )</td><td>data-sort-value=&quot;50000&quot;&gt; 50,000g</td><td>data-sort-value=&quot;650&quot;&gt; 650g</td></tr><tr><td/><td>Infinity Blade</td><td>17</td><td>The true form of the Galaxy Sword.</td><td>80-100</td><td>.02</td><td>Speed (+4) Defense (+2)</td><td>Combine the Galaxy Sword with Galaxy Soul (3) and Cinder Shard (60) in the Forge</td><td>N/A</td><td>data-sort-value=&quot;850&quot;&gt; 850g</td></tr><tr><td/><td>Haley&#x27;s Iron</td><td>6</td><td>It&#x27;s searing hot and smells like Haley&#x27;s hair.</td><td>30-45</td><td>.02</td><td>Speed (-1) Crit. Power (+10)</td><td>Sold by Haley for data-sort-value=&quot;70&quot;&gt; 70 Calico Eggs during the Desert Festival</td><td>N/A</td><td>data-sort-value=&quot;300&quot;&gt; 300g</td></tr><tr><td/><td>Leah&#x27;s Whittler</td><td>6</td><td>Leah&#x27;s favorite tool for shaping driftwood.</td><td>30-45</td><td>.02</td><td>Speed (-1) Crit. Power (+10)</td><td>Sold by Leah for data-sort-value=&quot;70&quot;&gt; 70 Calico Eggs during the Desert Festival</td><td>N/A</td><td>data-sort-value=&quot;300&quot;&gt; 300g</td></tr><tr><td/><td>Meowmere</td><td>4</td><td>An unusual weapon from a far away land...</td><td>20-20</td><td>.02</td><td>Speed (+4) Weight (+2)</td><td>Basement in Wizard&#x27;s Tower</td><td>N/A</td><td>data-sort-value=&quot;200&quot;&gt; 200g</td></tr></table>'"
test_propmpt = summarize_prompt.format(element=swords_table)

In [90]:
test_propmpt

'\nYou are an assistant tasked with summarizing the tables. \nGive a concise summary of the table or text.\n\nRespond only with the summary, no additionnal comment.\nDo not start your message by saying "Here is a summary" or anything like that.\nJust give the summary as it is.\n\nTable or text chunk: \'<table><tr><td>Image</td><td>Name</td><td>Level</td><td>Description</td><td>Damage</td><td>Critical Strike Chance</td><td>Stats</td><td>Location</td><td>Purchase Price</td><td>Sell Price</td></tr><tr><td/><td>Rusty Sword</td><td>1</td><td>A rusty, dull old sword.</td><td>2-5</td><td>.02</td><td>N/A</td><td>The Mines entrance</td><td>data-sort-value=&quot;250&quot;&gt; 250g</td><td>data-sort-value=&quot;50&quot;&gt; 50g</td></tr><tr><td/><td>Steel Smallsword</td><td>1</td><td>A standard metal blade.</td><td>4-8</td><td>.02</td><td>Speed (+2)</td><td>The Mines floor 20 chest reward</td><td>data-sort-value=&quot;750&quot;&gt; 750g</td><td>data-sort-value=&quot;50&quot;&gt; 50g</td></tr><tr>

In [91]:
tokens = encoding.encode(test_propmpt)
len(tokens)

3309

In [95]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": test_propmpt}]
)

result = response.choices[0].message.content
result

'The table provides details on various swords, including their names, levels, descriptions, damage, critical strike chances, stats, locations, purchase prices, and sell prices. The swords range from basic ones like the "Rusty Sword" with low damage and a purchase price of 250g, to advanced weapons such as the "Galaxy Sword" costing 50,000g. The information highlights where each weapon can be found or obtained, such as as rewards from specific mine levels or guild purchases. Stats include enhancements like speed, defense, or critical power, with critical strike chances mostly at .02, except for notable exceptions like the "Insect Head" and "Dark Sword" at .04.'

In [96]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[{"role": "user", "content": test_propmpt}]
)

result2 = response.choices[0].message.content
result2

'Table of 28 swords (Rusty Sword → Infinity Blade and Meowmere) showing name, level (1–17), short description, damage ranges (2–100), critical strike chance (mostly 0.02; some .04, .015 or 0), stat modifiers (Speed, Defense, Weight, Crit. Power/Chance, Vampiric, Crusader), source/location (primarily Mines floor chest rewards, Adventurer’s Guild, Skull Cavern, Volcano Dungeon, Desert Three Pillars, Forge, Fishing chests, Haunted Skull, Wizard’s Tower, festival vendors), purchase prices (many N/A; notable buys: Galaxy Sword 50,000g, Lava Katana 25,000g, Steel Falchion 9,000g, Insect Head 10,000g) and sell prices (50g–850g). Highlights: early cheap weapons (Rusty Sword, Steel Smallsword, Wooden Blade); mid-game powerful blades (Claymore, Obsidian Edge, Yeti Tooth, Steel Falchion); endgame/unique items (Galaxy Sword lv13 → craftable into Infinity Blade lv17, Dragontooth Cutlass, Dwarf Sword, Lava Katana); special effects include high Crit Power, Vampiric on Dark Sword, and festival-exclusi

In [97]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{"role": "user", "content": test_propmpt}]
)

result3 = response.choices[0].message.content
result3

'A catalog of 28 swords spanning levels 1–17, listing each weapon’s name, damage range, small crit chance (mostly 0.02), notable stats (speed, defense, weight, vampiric), and how to obtain it (various Mines chest rewards, Adventurer’s Guild, dungeon/treasure drops, Desert/Volcano/treasure chests, or vendor NPCs). Purchase prices range from about 50g to 50,000g, with typical sell prices around 50–250g; several high-value items include Galaxy Sword (50,000g) and insect-head-related weapons (10,000g buy).'